                                                Домашнє завдання N 5

За даними акселерометра з мобільного телефону потрібно класифікувати, якою діяльністю займається людина: йде, стоїть, біжить чи йде по сходах.

Використайте алгоритми SVM та випадковий ліс з бібліотеки scikit-learn. Як характеристики можете брати показники з акселерометра, проте щоб покращити результати роботи алгоритмів, спочатку можна підготувати наш датасет і розрахувати часові ознаки (time domain features).

Почнемо з імпорту потрібних бібліотек:

In [17]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

Давайте зараз подивимось які саме підпапки з даними в нас зберігаються у датасеті data:

In [18]:
data_dir_path = os.path.join('data')
data_files = os.listdir(data_dir_path)

data_files

['running', 'idle', '.DS_Store', 'walking', 'stairs']

Як бачимо, у піддиректорії "walking" знаходяться файли з даними, названі за шаблоном "walking-<номер>".csv. Це означає, що дані з акселерометра для кожної діяльності зберігаються у форматі CSV файлів, причому кожен файл відповідає окремій сесії або запису цієї діяльності:

In [19]:
sample_activity_dir = os.path.join(data_dir_path, 'walking')
sample_files = os.listdir(sample_activity_dir)

sample_files[:5]

['walking-576.csv',
 'walking-210.csv',
 'walking-204.csv',
 'walking-1429.csv',
 'walking-562.csv']

Прочитаємо один з csv-файлів, щоб зрозуміти, які саме дані нам надані:

In [20]:
sample_file_path = os.path.join(sample_activity_dir, 'walking-1.csv')
df = pd.read_csv(sample_file_path)

# Подивимося на перші 5 рядків датасету
df.head()

,accelerometer_X,accelerometer_Y,accelerometer_Z
0,2.322376,3.643975,11.827356
1,1.781286,4.668694,6.148310
2,7.963115,-4.113238,1.359907
3,1.996764,-1.398214,1.163582
4,5.185841,-12.143391,4.687847


Перш за все, потрібно об'єднати всі файли в єдиний датасет. Це включає завантаження даних з кожного файлу, додавання мітки класу (діяльності) до кожного рядка, та об'єднання цих даних в один DataFrame:

Спочатку напишемо функцію, яка буде зчитувати дані з кожної підпапки:

In [21]:
def load_data_from_directory(directory, label):
    files = os.listdir(directory)
    data_list = []
    for file in files:
        file_path = os.path.join(directory, file)
        data = pd.read_csv(file_path)
        data['Activity'] = label
        data_list.append(data)
    return pd.concat(data_list, ignore_index=True)

Далі зчитуємо дані з кожної підпапки активності та записуємо їх в єдиний датафрейм:

In [22]:
activities = ['idle', 'running', 'stairs', 'walking']
data_frames = []
for activity in activities:
    directory_path = os.path.join('data', activity)
    df = load_data_from_directory(directory_path, activity)
    data_frames.append(df)

complete_data = pd.concat(data_frames, ignore_index=True)

complete_data.shape, complete_data.head()

((193860, 4),
    accelerometer_X  accelerometer_Y  accelerometer_Z Activity
 0         0.009577         5.937620         7.570466     idle
 1         0.086191         6.555324         7.785944     idle
 2         0.004788         6.440403         7.139510     idle
 3         0.277727         6.430826         7.690176     idle
 4        -0.047884         6.239290         7.340623     idle)

Тепер потрібно нормалізувати дані, щоб можна було потім з ними працювати. Для цьго використаємо класс StandardScaler:

In [24]:
scaler = StandardScaler()
features = ['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']
complete_data[features] = scaler.fit_transform(complete_data[features])

complete_data

,accelerometer_X,accelerometer_Y,accelerometer_Z,Activity
0,-0.227723,0.347865,0.801712,idle
1,-0.218607,0.397385,0.831675,idle
2,-0.228292,0.388172,0.741787,idle
3,-0.195818,0.387404,0.818358,idle
4,-0.234559,0.372049,0.769752,idle
...,...,...,...,...
193855,-0.452192,-0.883975,-0.598540,walking
193856,-1.429830,-1.235984,-0.416767,walking
193857,-0.338248,-0.290895,2.896430,walking
193858,-0.204934,-1.132723,-0.197707,walking
